# Entity Extraction Model


In [16]:
import tqdm
import time
import contextlib



In [17]:
import logging as log

from tqdm import tqdm
from io import StringIO

class TqdmLogFormatter(object):
    def __init__(self, logger):
        self._logger = logger

    def __enter__(self):
        self.__original_formatters = list()

        for handler in self._logger.handlers:
            self.__original_formatters.append(handler.formatter)

            handler.terminator = ''
            formatter = log.Formatter( '%(message)s')
            handler.setFormatter(formatter)

        return self._logger

    def __exit__(self, exc_type, exc_value, exc_traceback):
        for handler, formatter in zip(self._logger.handlers, self.__original_formatters):
            handler.terminator = '\n'
            handler.setFormatter(formatter)

class TqdmLogger(StringIO):
    def __init__(self, logger):
        super().__init__()

        self._logger = logger

    def write(self, buffer):
        with TqdmLogFormatter(self._logger) as logger:
            logger.info(buffer)

    def flush(self):
        pass


In [18]:
import cohere
import pandas as pd
import requests
import datetime
import os,sys
sys.path.append('../scr')
#import config
#import tqdm
#import cProfile  
#import file_handler 
#import FileHandler
#file_data=FileHandler()
#api_key = conh_api.cohere_api["api_key"]

In [19]:
# Paste your API key here. Remember to not share publicly
#api from config

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(conh_api)

In [20]:
#data = file_data.dvc_get_data('../data/relations_dev.json', 'v3')
import json
#train data
#with open("../data/relations_dev.json",'r') as read_file:
train_df = pd.read_json(r"C:\Users\Genet Shanko\Prompt-Engineering-In-context-learning-with-GPT-3-and-LLMs\Data\relations_dev.json")
#test data
test_df = pd.read_json(r"C:\Users\Genet Shanko\Prompt-Engineering-In-context-learning-with-GPT-3-and-LLMs\Data\Test.json") 

In [21]:
train_df.head()

,document,tokens,relations
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG..."
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP..."
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG..."


In [22]:
test_df.head()

,document,tokens,relations
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D..."
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP..."
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX..."
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ..."
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ..."


In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   document   22 non-null     object
 1   tokens     22 non-null     object
 2   relations  22 non-null     object
dtypes: object(3)
memory usage: 656.0+ bytes


In [24]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   document   11 non-null     object
 1   tokens     11 non-null     object
 2   relations  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


In [25]:
def pre_process_tokens(tokens):
    enti={}
    for t in tokens:
        
        if t['entityLabel'] in enti.keys():
            enti[t['entityLabel']] = enti[t['entityLabel']] +','+t['text']
        else:
            enti[t['entityLabel']] = t['text']
    label = ""
    for k, v in enti.items():
        label += k+':'+v+"\n"
    return label

In [26]:
#pre-processing
train_df['label'] = train_df['tokens'].apply(pre_process_tokens)
train_df

,document,tokens,relations,label
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG...",DIPLOMA:Bachelor\nDIPLOMA_MAJOR:Mechanical Eng...
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP...","EXPERIENCE:10+ years,5+ years\nSKILLS:software..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP...",EXPERIENCE:3+ years\nSKILLS:Swift & Objective-C\n
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP...","EXPERIENCE:8+ years,5+ years\nSKILLS:software ..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG...",DIPLOMA:BS\nDIPLOMA_MAJOR:Computer Science\nEX...
5,8+ years of development experience with Python...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP...","EXPERIENCE:8+ years\nSKILLS:development,Python..."
6,Strong problem solving and analytical skills i...,"[{'text': '2+ years', 'start': 773, 'end': 781...","[{'child': 129, 'head': 125, 'relationLabel': ...",EXPERIENCE:2+ years\nSKILLS:performance modeli...
7,"Experience in backend services, systems, and d...","[{'text': 'BS/MS', 'start': 66, 'end': 71, 'to...","[{'child': 15, 'head': 11, 'relationLabel': 'D...",DIPLOMA:BS/MS\nDIPLOMA_MAJOR:Computer Science\...
8,experience: 5 + years of full-stack software d...,"[{'text': '5 + years', 'start': 12, 'end': 21,...","[{'child': 6, 'head': 2, 'relationLabel': 'EXP...","EXPERIENCE:5 + years,5+ years,5 + years,3+ yea..."
9,BS in Electrical Engineering or Computer Engin...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 2, 'head': 0, 'relationLabel': 'DEG...","DIPLOMA:BS,MS\nDIPLOMA_MAJOR:Electrical Engine..."


In [27]:
test_df['label'] = test_df['tokens'].apply(pre_process_tokens)
test_df

,document,tokens,relations,label
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:machine learning,..."
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...",EXPERIENCE:2+ years\nSKILLS:online advertising...
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","EXPERIENCE:5+ years,2+ years\nSKILLS:project m..."
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","DIPLOMA:MS,PhD\nDIPLOMA_MAJOR:Operations Resea..."
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","EXPERIENCE:12+ years\nSKILLS:research,Ph.D\nDI..."
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","EXPERIENCE:5+ years,3+ years,3+ years,5+ years..."


In [28]:
#entity extraction
train_word = []
for i in range(train_df.shape[0]):
    entity = train_df.label.iloc[i]
    file_doc = train_df.document.iloc[i].replace("\n"," ")
    if len(entity)!=0:
        train_word.append(file_doc+"\n\nExtracted Text:" +
                        '\n'+entity+"----\n")
#store the entity on the file
with open(r"C:\Users\Genet Shanko\Prompt-Engineering-In-context-learning-with-GPT-3-and-LLMs\Data\relations_dev.json",'w') as f:
    for item in train_word:
        # write each item on a new line
        f.write("%s\n" % item.strip())
    print('Successful')

Successful


In [49]:
test_doc = test_df.document.apply(
    lambda x: x.replace("\n", " ")+'\n\nExtracted Text:').to_list()

In [63]:
#The first thing is to extract all the data from our json
class cohereExtractor():
    def __init__(self, examples):
        self.examples = examples

    def make_prompt(self, example):
        prompt = self.examples + [example]
       
        return ("".join([str(exam) for exam in prompt]))

    def extract(self, example):
        extraction = co.generate(
            model='large',
            prompt=self.make_prompt(example),
            max_tokens=100,
            temperature=0.5,
            stop_sequences=["----"])
        
        return(extraction.generations[0].text[:-1])


cohereMovieExtractor = cohereExtractor(test_doc[:4])

In [64]:
print(cohereMovieExtractor.make_prompt(test_doc))

 Currently holding a faculty, industry, or government researcher position. Ph.D. and publications in machine learning, AI, computer science, statistics, applied mathematics, data science, or related technical fields. Experience leading a team in solving analytical problems using quantitative approaches. Experience manipulating and analyzing data from different sources. Experience in theoretical and empirical research and for answering questions with research. Ability to communicate research for public audiences of peers. Knowledge in a programming language. Ability to obtain and maintain work authorization in the country of employment in 2018.  PREFERRED  1+ year(s) of work experience in a university, industry, or government lab(s), in a role with primary emphasis on AI research. Experience driving original scholarship in collaboration with a team. First-author publications at peer-reviewed AI conferences (e.g. NIPS, CVPR, ICML, ICLR, ICCV, and ACL). Experience in developing and debugg

In [65]:
results = []
for text in tqdm(test_doc):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        print(extracted_text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

  9%|███████▌                                                                           | 1/11 [00:03<00:32,  3.28s/it]

 Currently enrolled in a full-time degree program and returning to the program after the completion of the internship. Publications or experience in fields related to in machine learning, AI, computer vision, natural language processing, computational neuroscience, optimization, computer science, statistics, applied mathematics, or data science. Experience solving analytical problems using quantitative approaches. Ability to manipulate and analyze large scale, high-dimensionality data from varying sources. Experience in utilizing theoretical and empirical research to solve problems. Ability to communicate research for publi


 18%|███████████████                                                                    | 2/11 [00:06<00:29,  3.23s/it]

 BA/BS 5+ years of program or project management experience 2+ years of technical project/program management experience Track record of operating independently Experience understanding user needs, gathering requirements, and defining scope Communication experience interacting with a variety of audiences from engineers, to vendors, to research leaders Track record of building cross-functional relationships  PREFERRED  Experience working with UX Research and/or UX Design

Extracted Text: Currently enrolled in a full-time degree program and returning to the program after th


 27%|██████████████████████▋                                                            | 3/11 [00:09<00:25,  3.21s/it]

...

 Bachelor's Degree in Computer Science, Information Systems, Engineering, or related field with a minimum of 1+ years of experience in the Information Technology field. Experience designing, implementing, and maintaining large, complex, enterprise-level applications. Experience with cloud-based applications. Experience with building and maintaining data pipelines. Experience with source code control systems. Experience with working with enterprise security requirements. Experience with full life cycle software development. Experience with agile software development methodologies. Experience with the use of test-driven developmen


 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [00:32<00:03,  3.27s/it]

 PhD with 3+ years, or MS with 5+ years of experience in a field emphasizing people research in organizations (e.g., Industrial/Organizational Psychology, Organizational Behavior, Labor Economics, Management, Policy Analysis, etc.) 3+ years of experience conducting quantitative people research in an organization or consulting environment, including working with stakeholders to understand and clarify their people research needs, and communicating analyses to technical and non-technical audiences Experience managing and analyzing structured and unstructured data Experience communicatin


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:35<00:00,  3.26s/it]

 BA/BS 5+ years of program or project management experience 2+ years of technical project/program management experience Track record of operating independently Experience understanding user needs, gathering requirements, and defining scope Communication experience interacting with a variety of audiences from engineers, to vendors, to research leaders Track record of building cross-functional relationships  PREFERRED  Experience working with UX Research and/or UX Design

Extracted Text: Currently enrolled in a full-time degree program and returning to the program after th


In [53]:
pd.DataFrame(data={'text': test_doc, 'extracted_text': results})

,text,extracted_text
0,"Currently holding a faculty, industry, or gov...",BA/BS 5+ years of program or project manageme...
1,2+ years experience in the online advertising...,BA/BS 5+ years of program or project manageme...
2,BA/BS 5+ years of program or project manageme...,Currently enrolled in a full-time degree prog...
3,Currently enrolled in a full-time degree prog...,Currently enrolled in a full-time degree prog...
4,Currently enrolled in a full-time degree prog...,Currently enrolled in a full-time degree prog...
5,MS in a quantitative field such as Operations...,MS in a quantitative field such as Operations...
6,"Ph.D. with 5+ years of experience, MS with 7+...",Ph.D. or equivalent experience. 7+ years of e...
7,Graduating with a Ph.D. in Computer Science o...,Graduating with a Ph.D. in Computer Science o...
8,"12+ years of research experience, in-house or...",PhD in Computer Science or related field. 10+...
9,"PhD with 3+ years, or MS with 5+ years of exp...",PhD in a quantitative field such as Operation...


In [55]:
def clean_extracted(x):
    x = "".join(x)
    x = x.lower()
    x = x[:-3]
    x = x.lstrip()
    
    x = x.split("\n")
    x = "".join(x)
    x = x.split(':')
    x = "".join(x)
    return x

In [56]:
def clean_test(x):
    x = "".join(x)
    x = x.lower()
    x = x.lstrip()
    x = x.split("\n")
    x = "".join(x)
    x = x.split(':')
    x = "".join(x)
    return x

In [57]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['document'].apply(
            lambda x: "Job: "+x.replace("\n", " ")+'\n\nExtracted Text:').to_list()):
        extracted.append(str(i).strip())

In [58]:
# Save results
test_df['extracted_text'] = extracted
test_df['extracted_text'] = test_df['extracted_text'].apply(clean_extracted)
test_df['label'] = test_df['label'].apply(clean_test)
from difflib import SequenceMatcher
score = []
for i in range(test_df.shape[0]):
    
    score.append(SequenceMatcher(None,test_df['label'].iloc[i],
                          test_df['extracted_text'].iloc[i]).ratio())
# Compare the label to the extracted text
test_df['similarity_score'] = pd.DataFrame(score)

# Print the accuracy
print(f'Classification accuracy {test_df["similarity_score"].mean() *100}%')

Classification accuracy 4.066765259547664%


In [59]:
test_df[test_df['similarity_score']==0]

,document,tokens,relations,label,extracted_text,similarity_score
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","diplomams,phddiploma_majoroperations research,...",,0.0
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],,extracted textextracted textextracted textextr...,0.0
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",diplomaph.d.diploma_majorcomputer science,ph.d. in computer science or related field. 7...,0.0
